In [1]:
import requests
import xmltodict
import json
import pandas as pd
import time
import matplotlib.pyplot as plt
from matplotlib import font_manager,rc
from datetime import datetime
import seaborn as sns

today = datetime.today().strftime("%Y%m%d") #오늘

#수출입 데이터를 표로 보여줌
def itemtrade(serviceKey,strtYymm,endYymm,hsSgn):

    url = '	http://apis.data.go.kr/1220000/Itemtrade/getItemtradeList'
    params = {'serviceKey': serviceKey,'strtYymm':strtYymm,'endYymm':endYymm,'hsSgn':hsSgn}
    response = requests.get(url,params=params)

    dic = xmltodict.parse(response.content)
    # print(dic)
    jsonString = json.dumps(dic['response']['body']['items'])
    json_object = json.loads(jsonString)
      
    col = ['품목','HSCODE','기간','수출금액','수출중량','수입금액','수입중량']

    df = []
    for item in json_object['item']:
        statKor = item['statKor'] # 폼목명
        hsCode = item['hsCode'] #hscode
        year = item['year'] #기간
        expDlr = item['expDlr'] #수출금액
        expWgt = item['expWgt'] #수출중량
        impDlr = item['impDlr'] #수입금액
        impWgt = item['impWgt'] #수입중량
        df.append([statKor,hsCode,year,expDlr,expWgt,impDlr,impWgt])

    table = pd.DataFrame(df,columns=col)
    table = table.drop(table[table['기간'] == '총계'].index)
    table['수출금액'] = table['수출금액'].apply(pd.to_numeric,errors = 'ignore')
    table['수출중량'] = table['수출중량'].apply(pd.to_numeric,errors = 'ignore')
    table['수출단가'] = round(table['수출금액']/table['수출중량'],1)
    table['연도'] = table['기간'].str.split('.').str[0]
    table['월'] = table['기간'].str.split('.').str[1]+'월'
    
    time.sleep(3)
    return table

#수출입 merge 한 데이터 groupby 로 변경
def dataframe_pivottable(table,title,hscode):
    pv_table = pd.pivot_table(table,values='수출금액',index='월',columns='연도',aggfunc='max')
    pv_table['YoY'] = round(((pv_table['2024']-pv_table['2023'])/pv_table['2023'])*100,1)
    return pv_table
    
def graph(dataframe,title,hscode):
    # 기본 스타일 설정

    font_path='./malgun.ttf'
    font_name=font_manager.FontProperties(fname=font_path).get_name()
    rc('font',family=font_name)
    plt.rc('axes',unicode_minus=False)
    
    # Figure와 축 생성
    fig, ax1 = plt.subplots(figsize=(6, 3.5))
    palette = {
        '2023': 'lightgreen',  # 연한 파란색lightblue
        '2024': 'red'  # 연한 초록색
    }
    # 수출금액에 대한 barplot 생성
    sns.barplot(
        data=dataframe, 
        x='월', 
        y='수출금액', 
        hue='연도', 
        palette=palette, 
        alpha = 0.5,
        ax=ax1,
        edgecolor = 'lightgrey',
        zorder=2  # 막대그래프의 zorder를 낮게 설정
    )
    
    # 수출금액을 위한 첫 번째 y축 라벨 설정
    ax1.set_ylabel('수출금액 (원)', fontsize=10)
    
    # 두 번째 y축 생성
    ax2 = ax1.twinx()
    
    # YoY를 꺾은선 그래프로 그리기
    sns.lineplot(
        data=dataframe, 
        x='월', 
        y='YoY', 
        color='blue',  # 연한 빨강색
        marker='o', 
        markersize=6,
        markerfacecolor='blue',  # 마커 내부 하얀색
        markeredgewidth=1.2,  # 마커 테두리 두께
        markeredgecolor='blue',  # 마커 테두리 진한 빨강색
        linewidth=2,  # 꺾은선 두께 얇게
        alpha = 0.6,
        ax=ax2
    )
    
    # YoY를 위한 두 번째 y축 라벨 설정
    ax2.set_ylabel('YoY(%)', fontsize=10)
    
    # 범례 설정 - 타이틀 제거, 막대와 꺾은선 그래프를 명확히 구분
    ax1.legend(loc='upper left', fontsize=10, title='')  # 범례 크기와 타이틀 제거
    ax2.legend(["24'YoY(%)"], loc='upper right', fontsize=10)
    
    # 그래프 제목과 공백 추가
    plt.title(f'{title}_{hscode}', fontsize=10)
    
    # X축 라벨 제거 (월이라는 스케일 이름 제거)
    ax1.set_xlabel('')
    
    # 그래프 높이를 2/3까지만 표시
    ax1.set_ylim(0, dataframe['수출금액'].max() * 1.3)
    # 오른쪽 y축 스케일 조정
    yoy_min = dataframe['YoY'].min()
    yoy_max = dataframe['YoY'].max()

    # 여유를 두기 위해 범위 설정
    if yoy_min < 0 and yoy_max > 0:
        ax2.set_ylim(yoy_min * 1.2, yoy_max * 1.2)
    elif yoy_min >= 0:
        ax2.set_ylim(yoy_min * 0.8, yoy_max * 1.2)
    else:
        ax2.set_ylim(yoy_min * 1.2, yoy_max * 0.8)
    
    # X축 및 Y축 눈금 크기 조정
    ax1.tick_params(axis='x', labelsize=10)
    ax1.tick_params(axis='y', labelsize=10)
    ax2.tick_params(axis='y', labelsize=10)
    
    # 그리드 제거
    ax1.grid(True, which='both', axis='both', color='#D3D3D3', linestyle='-', linewidth=0.5,zorder=2)
    ax2.grid(False)
    
    # 배경색 설정 및 그래프 레이아웃 조정
    fig.patch.set_facecolor('white')
    plt.tight_layout()
    
    # 그래프 보여주기
    plt.show()

#수출입 저장
def save_excel(table,title):
    file_name = f'C:\투자기록\정리\코딩\장내매수\hscode_excel\수출입_종합.xlsx'

    writer = pd.ExcelWriter(file_name, mode='a', engine='openpyxl', if_sheet_exists='overlay')
    print(writer.sheets[title])
        
    #     if max_row == 1:
    #         table.to_excel(
    #             writer, 
    #             sheet_name=title,
    #             startcol = 0,
    #             startrow = 0,
    #             index=False,
    #             encoding = 'utf-8',
    #             na_rep = '',      # 결측값을 ''으로 채우기
    #             inf_rep = '',     # 무한값을 ''으로 채우기
    #             # header = None
    #             )
    #         writer.save()
    #         writer.close()
    #     else:
    #         table.to_excel(
    #             writer, 
    #             sheet_name=title,
    #             startcol = 0,
    #             startrow = writer.sheets[title].max_row,
    #             index=False,
    #             encoding = 'utf-8',
    #             na_rep = '',      # 결측값을 ''으로 채우기
    #             inf_rep = '',     # 무한값을 ''으로 채우기
    #             header = None
    #             )
    #         writer.save()
    #         writer.close()
    # except:
    #     table.to_excel(
    #         excel_writer = file_name,
    #         sheet_name = title,
    #         index = False,       # 0부터 시작하는 자연수 인덱스는 의미가 없음.
    #         # columns = ['col1, 'col2', 'col3'],
    #         encoding = 'utf-8',
    #         na_rep = '',      # 결측값을 ''으로 채우기
    #         inf_rep = ''     # 무한값을 ''으로 채우기
    #     )     # 해당 파일이 열려있으면 안됨.            
    # writer.close()

if __name__ == '__main__':
    serviceKey='TVTYfCAg2gU/4zSSZm3Lny4s3G8DPFqBkDN6mcSc1s7EWq0vS2gXzqCZjtDL6OIzj3Tu6yKZm8Fx4ToS+ZQYsw=='
    start = ['201801','201901','202001','202101','202201','202301','202401'] # '202201','202301','
    end = ['201812','201912','202012','202112','202212','202312','202412'] #'202212','202312',

    # start = ['201801','201901','202001','202101'] # '202201','202301','
    # end = ['201812','201912','202012','202112'] #'202212','202312',
    
    hsSgn =['9018909080']
    # '2931909090' #TMA, 레이크머티리얼즈
    # '3304991000'# 기초화장품
    # '9018909080' 에스테틱 의료기기 장비
    # '9018909090' 에스테틱 의료기기 부품
    # ,'3304991000','1902301010','3005904000','3304999000','8506500000','3204901010','3304992000'
    title = ''
    hscode = ''
    ttl = pd.DataFrame()
    for h_code in hsSgn:
        for strtYymm,endYymm in zip(start,end):
            try:
                temp_ttl = itemtrade(serviceKey,strtYymm,endYymm,h_code)
                title = temp_ttl.iloc[0,0]
                hscode = temp_ttl.iloc[0,1]
                ttl = pd.concat([ttl,temp_ttl])
            except Exception as e:
                print(strtYymm,e)
                
    ttl.to_excel(f'{hsSgn}_{today}.xlsx')
    if '수출금액' in ttl.columns:
        ttl['YoY'] = ttl['수출금액'].pct_change(12)
        ttl['YoY'] = round(ttl['YoY']*100,1)
    else:
        print('다시')
    # save_excel(ttl,title)
    # dd = dataframe_pivottable(ttl,title,hscode)
    # graph(ttl,title,hscode)

    # graph(pv_ta)


202301 'NoneType' object is not subscriptable
202401 'NoneType' object is not subscriptable
